# Notes:
##### This notebook demonstrates the preliminary method used to parse JSON from 2 web apis from a COVID-19 site. The data was then inserted into our PostgreSQL database instance on AWS RDS. Since then, the process is was migrated to Flask to include more streamlining through creation of classes for each table and a more organized architecture for our API services, unit tests, models, and images. In the end, the Flask app was primarily replaced by AWS Lambda functions.

In [ ]:
# Imports
import psycopg2
import requests
from psycopg2.extras import Json

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
# You will need to get these credentials
host = 'host'
user = 'user'
password = 'password'

In [ ]:
# Define our function to insert JSON data into database. You will need this 
# function to quickly insert data!
def db_insertion():
    curs = conn.cursor()
    if curs != None:
        try:
            curs.execute( sql_string )
            conn.commit()
            print('\nfinished INSERT INTO execution')
        except Exception as error:
            print("\nexecute_sql() error:", error)
            conn.rollback()
        #close cursor and connection
        curs.close()

In [ ]:
##########################summary TABLE#######################################

In [ ]:
# Connect to AWS RDS PostgreSQL DB instance via Psycopg - with secret password
conn = psycopg2.connect(host=host, user=user, password=password)

In [ ]:
# Initial Table Creation
# drop duplicates
drop_table = 'DROP TABLE IF EXISTS summary;'

# create table query
create_table = '''
CREATE TABLE summary (
countrycode TEXT,
slug TEXT PRIMARY KEY,
newconfirmed INTEGER,
totalconfirmed INTEGER,
newdeaths INTEGER,
totaldeaths INTEGER,
newrecovered INTEGER,
totalrecovered INTEGER,
date TIMESTAMP
)'''

creation_queries = [drop_table, create_table]

# Run queries
for query in creation_queries:
    c = conn.cursor()
    c.execute(query)
    conn.commit()
    c.close()

In [ ]:
# Put in that sweet data request
response = requests.get("https://api.covid19api.com/summary")
list_of_dict = response.json()

# Just get country data
record_list = list_of_dict['Countries']

# Separate keys and values
columns = [(list(x.keys()))[1:] for x in record_list][0]
values = [(list(x.values()))[1:] for x in record_list]

In [ ]:
columns

['CountryCode',
 'Slug',
 'NewConfirmed',
 'TotalConfirmed',
 'NewDeaths',
 'TotalDeaths',
 'NewRecovered',
 'TotalRecovered',
 'Date']

In [ ]:
values[5]

['AD', 'andorra', 4, 717, 1, 37, 13, 248, '2020-04-21T03:15:26Z']

In [ ]:
values[20]

['BY', 'belarus', 1485, 6264, 4, 51, 20, 514, '2020-04-21T03:15:26Z']

In [ ]:
# Use Python to parse the JSON object 

# value string for the SQL string
values_str = ''

# enumerate over records' values
for i, record in enumerate(values):
    #declare empty list for values
    val_list = []

    # append each value to a new list of values
    for v, val in enumerate(record):
        if type(val) == str:
            val = str(Json(val)).replace('"','')
        val_list += [ str(val) ]

    # put parentheses around each record string
    values_str += "(" + ', '.join( val_list ) + "),\n"

# remove the last comma and end SQL with a semicolon
values_str = values_str[:-2] + ";"

# Declare an SQL string for postgresql records

# concatenate the SQL string
table_name = "summary"
sql_string = "INSERT INTO %s (%s)\nVALUES %s" % (
    table_name,
    ', '.join(columns),
    values_str
)

len(sql_string)

17180

In [ ]:
# Insert JSON data into db
db_insertion()


finished INSERT INTO execution


In [ ]:
# Close our connection 
conn.close()

In [ ]:
##########################uscounty TABLE#######################################

In [ ]:
# Connect to AWS RDS PostgreSQL DB instance via Psycopg - with secret password
conn = psycopg2.connect(host=host, user=user, password=password)

In [ ]:
# Initial Table Creation
# drop duplicates
drop_table = 'DROP TABLE IF EXISTS uscounties;'

# create table query
create_table = '''
CREATE TABLE uscounties (
id SERIAL PRIMARY KEY,
country TEXT,
countrycode TEXT,
province TEXT,
city TEXT,
citycode TEXT,
lat REAL,
lon REAL,
cases INTEGER,
status TEXT,
date TIMESTAMP
)'''

creation_queries = [drop_table, create_table]

# Run queries
for query in creation_queries:
    c = conn.cursor()
    c.execute(query)
    conn.commit()
    c.close()

In [ ]:
# Put in that sweet data request
response = requests.get("https://api.covid19api.com/country/us/status/confirmed/live")
record_list = response.json()

# Separate keys and values
columns = [(list(x.keys())) for x in record_list][0]
values = [(list(x.values())) for x in record_list]

In [ ]:
columns

['Country',
 'CountryCode',
 'Province',
 'City',
 'CityCode',
 'Lat',
 'Lon',
 'Cases',
 'Status',
 'Date']

In [ ]:
values[20]

['United States of America',
 'US',
 'Wyoming',
 'Crook',
 '56011',
 '44.59',
 '-104.57',
 0,
 'confirmed',
 '2020-01-22T00:00:00Z']

In [ ]:
# Use Python to parse the JSON object 

# value string for the SQL string
values_str = ''

# enumerate over records' values
for i, record in enumerate(values):
    #declare empty list for values
    val_list = []

    # append each value to a new list of values
    for v, val in enumerate(record):
        if type(val) == str:
            val = str(Json(val)).replace('"','')
        val_list += [ str(val) ]

    # put parentheses around each record string
    values_str += "(" + ', '.join( val_list ) + "),\n"

# remove the last comma and end SQL with a semicolon
values_str = values_str[:-2] + ";"

# Declare an SQL string for postgresql records

# concatenate the SQL string
table_name = "uscounties"
sql_string = "INSERT INTO %s (%s)\nVALUES %s" % (
    table_name,
    ', '.join(columns),
    values_str
)

len(sql_string)

37571429

In [ ]:
# Insert JSON data into db
db_insertion()


finished INSERT INTO execution


In [ ]:
# Close our connection 
conn.close()